- https://learn.microsoft.com/en-us/training/modules/intro-audio-classification-pytorch/1-introduction

- https://medium.com/@lelandroberts97/musical-genre-classification-with-convolutional-neural-networks-ff04f9601a74

- https://towardsdatascience.com/audio-classification-with-deep-learning-in-python-cf752b22ba07

- https://pytorch.org/audio/main/index.html

- https://www.scaler.com/topics/pytorch/torchaudio-in-pytorch/

- https://en.wikipedia.org/wiki/Spectrogram

- https://towardsdatascience.com/audio-deep-learning-made-simple-part-2-why-mel-spectrograms-perform-better-aad889a93505

- https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5

In [1]:
import os
import yaml
#
import torch
import torch.nn as nn
# import torch.nn.
from torch.utils.data import Dataset, DataLoader, random_split
#
import torchaudio
# import torchaudio.transforms as T
# import torchaudio.functional as F
#
from torchvision.io import read_image
from torchvision.transforms import ConvertImageDtype
#
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
#
from sklearn.model_selection import train_test_split

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [3]:
cache_file = 'data.yaml'
data_path = 'data/processed_spectrograms_with_standardization/'

In [4]:
cache = yaml.safe_load(open(cache_file, 'r'))

In [5]:
data = [(el, float(cache[el]['stats'] >= 38.225)) for el in cache if cache[el]['year'] == 2022]
len(data)

104

In [6]:
X, y = zip(*data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [7]:
class Podcast(Dataset):
    
    def __init__(self, X, y):
        self.x = X
        self.y = y
        self.cid = ConvertImageDtype(torch.float32)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        label = self.y[idx]
        episode = self.x[idx]
        image = read_image(f'{data_path}/{episode}.png')
        return self.cid(image), label

In [8]:
train = Podcast(X_train, y_train)
train_dataloader = DataLoader(train)

In [9]:
test = Podcast(X_test, y_test)
test_dataloader = DataLoader(test)

# Model

In [43]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(4, 8, kernel_size=8, stride=2),
            nn.AvgPool2d(kernel_size=4),
            nn.ReLU(),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=16, stride=2),
            nn.AvgPool2d(kernel_size=8),
            nn.ReLU(),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=32, stride=2),
            nn.AvgPool2d(kernel_size=16),
            nn.ReLU(),
        )
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.33)
        self.linear1 = nn.Linear(256, 32)
        self.linear2 = nn.Linear(32, 1)


    def forward(self, x):
        x = self.layer1(x)
        print(x.shape)
        x = self.layer2(x)
        print(x.shape)
        x = self.layer3(x)
        print(x.shape)
        #    
        x = self.flatten(x)
        print(x.shape)
        x = nn.functional.relu(self.linear1(x))
#         x = self.dropout(x, training=self.training)
        print(x.shape, self.training)
        #
        return self.linear2(x)  

model = CNNet().to(device, dtype=torch.float32)

In [36]:
model

CNNet(
  (layer1): Sequential(
    (0): Conv2d(4, 8, kernel_size=(8, 8), stride=(2, 2))
    (1): AvgPool2d(kernel_size=4, stride=4, padding=0)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(16, 16), stride=(2, 2))
    (1): AvgPool2d(kernel_size=8, stride=8, padding=0)
    (2): ReLU()
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(32, 32), stride=(2, 2))
    (1): AvgPool2d(kernel_size=16, stride=16, padding=0)
    (2): ReLU()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.3, inplace=False)
  (linear1): Linear(in_features=256, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)

In [37]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [51]:
y_score = []
y_true = []

model.train()
for idx, (X, y) in enumerate(train_dataloader):

    X, y = X.to(device), y.to(device)
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output[0], y)
    #
    print(output.view(1), y)
    print(loss)
    y_score.append(loss.item())
    y_true.append(y.item())
    #
    loss.backward()
    optimizer.step()

torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0032], grad_fn=<ViewBackward0>) tensor([1.], dtype=torch.float64)
tensor(0.6947, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0031], grad_fn=<ViewBackward0>) tensor([0.], dtype=torch.float64)
tensor(0.6916, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0030], grad_fn=<ViewBackward0>) tensor([0.], dtype=torch.float64)
tensor(0.6916, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.S

torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0031], grad_fn=<ViewBackward0>) tensor([0.], dtype=torch.float64)
tensor(0.6916, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0030], grad_fn=<ViewBackward0>) tensor([0.], dtype=torch.float64)
tensor(0.6916, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0032], grad_fn=<ViewBackward0>) tensor([1.], dtype=torch.float64)
tensor(0.6947, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.S

torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0032], grad_fn=<ViewBackward0>) tensor([0.], dtype=torch.float64)
tensor(0.6916, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0030], grad_fn=<ViewBackward0>) tensor([0.], dtype=torch.float64)
tensor(0.6916, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) True
tensor([-0.0032], grad_fn=<ViewBackward0>) tensor([1.], dtype=torch.float64)
tensor(0.6947, dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.S

In [52]:
roc_auc_score(y_true, y_score)

1.0

In [54]:
y_score = []
y_true = []

model.eval()
with torch.no_grad():
    for X, y in test_dataloader:
        print('.', end='')
        #
        X, y = X.to(device), y.to(device)
        output = model(X)
        #
        y_score.append(
            criterion(output[0], y).item()
        )
        y_true.append(y.item())
        #
        print(y_score[-1], output, y)
#         test_loss += cost(pred, Y).item()
#         correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

.torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) False
0.691569983959198 tensor([[-0.0032]]) tensor([0.], dtype=torch.float64)
.torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) False
0.6916375160217285 tensor([[-0.0030]]) tensor([0.], dtype=torch.float64)
.torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) False
0.694721382111311 tensor([[-0.0031]]) tensor([1.], dtype=torch.float64)
.torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) False
0.694707565009594 tensor([[-0.0031]]) tensor([1.], dtype=torch.float64)
.torch.Size([1, 8, 2755, 1846])
torch.Size([1, 16, 171, 114])
torch.Size([1, 32, 4, 2])
torch.Size([1, 256])
torch.Size([1, 32]) False
0.691636860370636 tensor([[-0.0030]]

In [47]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [56]:
roc_auc_score(y_true, y_score)

1.0

In [57]:
average_precision_score(y_true, y_score)

1.0

In [14]:
def train(dataloader, model, criterion, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):

        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output.view(1), Y.type(torch.FloatTensor))
        #
        print(output.view(1), Y)
        print(loss)
        #
        loss.backward()
        optimizer.step()
        
#         print(loss)
#         if batch % 100 == 0:
#             current = batch * len(X)
#             print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')

In [ ]:
model.eval()
test_loss, correct = 0, 0
class_map = ['no', 'yes']

with torch.no_grad():
    for batch, (X, Y) in enumerate(test_dataloader):
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        print("Predicted:\nvalue={}, class_name= {}\n".format(pred[0].argmax(0),class_map[pred[0].argmax(0)]))
        print("Actual:\nvalue={}, class_name= {}\n".format(Y[0],class_map[Y[0]]))
        break

In [ ]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, Y) in enumerate(dataloader):
            X, Y = X.to(device), Y.to(device)
            pred = model(X)

            test_loss += cost(pred, Y).item()
            correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    print(f'\nTest Error:\nacc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}\n')

In [15]:
epochs = 15

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, criterion, optimizer)
    test(test_dataloader, model)
print('Done!')

Epoch 1
-------------------------------
580 torch.Size([4, 22051, 14776])
torch.Size([1, 128, 340, 226])
torch.Size([1, 9835520])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x9835520 and 739840x64)

- https://wandb.ai/ayush-thakur/dl-question-bank/reports/How-to-Handle-Images-of-Different-Sizes-in-a-Convolutional-Neural-Network--VmlldzoyMDk3NzQ

- https://blog.paperspace.com/global-pooling-in-convolutional-neural-networks/

In [52]:
import random

In [65]:
probas = [0.1, 0.4, 0.2, 0.3]
size = 10
result = []

res = [0]
for p in probas:
    res.append(res[-1]+p)
        
# res
for _ in range(size):
#     print(random.random())
    rr = random.random()
    print(rr)
    for idx in range(len(res)-1):
        print(res[idx], res[idx+1], result)
        if rr >= res[idx] and rr < res[idx+1]:
            print(idx)
            result.append(idx)
            break
            
print(res, result)

0.03358662964523873
0 0.1 []
0
0.3405022703178888
0 0.1 [0]
0.1 0.5 [0]
1
0.12637400239713015
0 0.1 [0, 1]
0.1 0.5 [0, 1]
1
0.7956959568389843
0 0.1 [0, 1, 1]
0.1 0.5 [0, 1, 1]
0.5 0.7 [0, 1, 1]
0.7 1.0 [0, 1, 1]
3
0.021017163545093864
0 0.1 [0, 1, 1, 3]
0
0.2790766399824478
0 0.1 [0, 1, 1, 3, 0]
0.1 0.5 [0, 1, 1, 3, 0]
1
0.19695354310223356
0 0.1 [0, 1, 1, 3, 0, 1]
0.1 0.5 [0, 1, 1, 3, 0, 1]
1
0.1389510492928656
0 0.1 [0, 1, 1, 3, 0, 1, 1]
0.1 0.5 [0, 1, 1, 3, 0, 1, 1]
1
0.3078099601608696
0 0.1 [0, 1, 1, 3, 0, 1, 1, 1]
0.1 0.5 [0, 1, 1, 3, 0, 1, 1, 1]
1
0.5177160995077268
0 0.1 [0, 1, 1, 3, 0, 1, 1, 1, 1]
0.1 0.5 [0, 1, 1, 3, 0, 1, 1, 1, 1]
0.5 0.7 [0, 1, 1, 3, 0, 1, 1, 1, 1]
2
[0, 0.1, 0.5, 0.7, 1.0] [0, 1, 1, 3, 0, 1, 1, 1, 1, 2]


In [51]:
seq = [1, 1, 0, 0, 1, 1]
# seq = [1, 1, 0, 0, 1, 1, 1]
# seq = [0, 0, 0]
# seq = []
res = 0
tmp = 0
for el in seq:
    if el == 1:
        tmp += 1
        res = max(res, tmp)
    else:
        tmp = 0
    
    
print(res)

2


In [40]:
len(podcast)

78

In [12]:
yaml.safe_load(open(cache_file, 'r'))[571]

{'day': 3,
 'dow': 'Tuesday',
 'month': 'May',
 'year': 2022,
 'title': 'SDS 571: Collaborative, No-Code Machine Learning',
 'duration': 58,
 'link': 'https://www.youtube.com/embed/c5LyEtZcxiY',
 'stats': 41.5}

# Links

- https://github.com/daisukelab/ml-sound-classifier

- https://www.kaggle.com/c/freesound-audio-tagging

- https://github.com/daisukelab/sound-clf-pytorch

- https://clear.ml/blog/audio-classification-with-pytorchs-ecosystem-tools

- https://www.kaggle.com/code/aayush9753/audio-classification-starter-in-pytorch

- https://blogs.rstudio.com/ai/posts/2022-10-06-audio-classification-torch/

- https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/audio_classifier_tutorial.ipynb

- https://bamblebam.medium.com/audio-classification-and-regression-using-pytorch-48db77b3a5ec

- https://www.kaggle.com/code/psycheshaman/pytorch-human-speech-classification

- https://huggingface.co/docs/transformers/tasks/audio_classification

- https://github.com/huggingface/transformers/blob/main/examples/pytorch/audio-classification/README.md

- https://music-classification.github.io/tutorial/part3_supervised/tutorial.html

- https://medium.com/@mlg.fcu/using-python-to-classify-sounds-a-deep-learning-approach-ef00278bb6ad

- https://www.youtube.com/watch?v=TkwXa7Cvfr8&t=1251s

- https://www.youtube.com/watch?v=O5xeyoRL95U

- https://medium.com/mlearning-ai/music-genre-classification-using-cnn-part-2-classification-ee5400cfbc4f

In [41]:
podcast[0]

610 torch.Size([4, 22051, 662])


(tensor([[[201, 188, 251,  ..., 255, 255, 255],
          [220, 192, 241,  ..., 242, 255, 255],
          [213, 193, 234,  ..., 235, 255, 255],
          ...,
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255]],
 
         [[201, 188, 251,  ..., 255, 255, 255],
          [220, 192, 241,  ..., 242, 255, 255],
          [213, 193, 234,  ..., 235, 255, 255],
          ...,
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255]],
 
         [[201, 188, 251,  ..., 255, 255, 255],
          [220, 192, 241,  ..., 242, 255, 255],
          [213, 193, 234,  ..., 235, 255, 255],
          ...,
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 255, 255, 255]],
 
         [[255, 255, 255,  ..., 255, 255, 255],
          [255, 255, 255,  ..., 25

In [19]:
cache = yaml.safe_load(open('data.yaml', 'r'))

In [20]:
popularity_dict = {el: cache[el]['stats'] for el in cache if cache[el]['year'] == 2022}
len(popularity_dict)

104

In [21]:
sorted(popularity_dict.items(), key=lambda x: x[1])[0]

(638, 20.0)

In [22]:
sorted(popularity_dict.items(), key=lambda x: x[1])[-1]

(537, 48.7)

# Sources

- https://speechprocessingbook.aalto.fi/Representations/Melcepstrum.html

- https://mmchiou.gitbooks.io/uwsgi/content/tensorflow-speech-recognition-challenge/kernel/speech-representation-and-data-exploration/notebooksrde/visualizationspae.html

- https://vtiya.medium.com/mfcc-vs-mel-spectrogram-8f1dc0abbc62

- https://ketanhdoshi.github.io/Audio-ASR/

- https://towardsdatascience.com/audio-deep-learning-made-simple-part-2-why-mel-spectrograms-perform-better-aad889a93505

- https://www-i6.informatik.rwth-aachen.de/publications/download/937/T%7Bu%7DskeZolt%7Ba%7DnGolikPavelSchl%7Bu%7DterRalfNeyHermann--AcousticModelingwithDeepNeuralNetworksUsingRawTimeSignalfor%7BLVCSR%7D--2014.pdf
